From Tensor SkFlow: https://github.com/tensorflow/tensorflow/blob/master/tensorflow/examples/skflow/mnist.py

## Import

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from sklearn import metrics
from tensorflow.contrib import learn

import chainer.functions as F
import chainer.links as L
from chainer import optimizers, Chain
from commonml.sklearn import ChainerEstimator, SoftmaxCrossEntropyClassifier

import logging
logging.basicConfig(format='%(levelname)s : %(message)s', level=logging.INFO)
logging.root.level = 20

## Download and load MNIST data.

In [ ]:
mnist = learn.datasets.load_dataset('mnist')

## Linear classifier.

In [ ]:
class LinearModel(Chain):

    def __init__(self, in_size):
        super(LinearModel, self).__init__(l1=L.Linear(in_size, 10))

    def __call__(self, x):
        h1 = self.l1(x)
        return h1

classifier = ChainerEstimator(model=SoftmaxCrossEntropyClassifier(LinearModel(mnist.train.images.shape[1])),
                              optimizer=optimizers.AdaGrad(lr=0.01),
                              batch_size=100,
                              gpu=0,
                              n_epoch=100)
classifier.fit(mnist.train.images, mnist.train.labels)
score = metrics.accuracy_score(mnist.test.labels, classifier.predict(mnist.test.images))
print('Accuracy: {0:f}'.format(score))

## Convolutional network

In [ ]:
class Model(Chain):

    def __init__(self):
        super(Model, self).__init__(conv1=F.Convolution2D(1, 32, 5),
                                    conv2=F.Convolution2D(32, 64, 5),
                                    l3=L.Linear(64*4*4, 1024),
                                    l4=L.Linear(1024, 10),
                                   )

    def __call__(self, x, train=True):
        h1 = F.max_pooling_2d(F.relu(self.conv1(x)), 2)
        h2 = F.max_pooling_2d(F.relu(self.conv2(h1)), 2)
        h3 = F.dropout(F.relu(self.l3(h2)), ratio=.5, train=train)
        h4 = self.l4(h3)
        return h4

classifier = ChainerEstimator(model=SoftmaxCrossEntropyClassifier(Model()),
                              optimizer=optimizers.AdaGrad(lr=0.001),
                              batch_size=100,
                              gpu=0,
                              n_epoch=100)

classifier.fit(mnist.train.images.reshape((len(mnist.train.images), 1, 28, 28)), mnist.train.labels)
score = metrics.accuracy_score(mnist.test.labels, classifier.predict(mnist.test.images.reshape((len(mnist.test.images), 1, 28, 28))))
print('Accuracy: {0:f}'.format(score))